In [1]:
!git clone https://github.com/miguel-fc/Neutron_Reflect.git

fatal: destination path 'Neutron_Reflect' already exists and is not an empty directory.


In [2]:
cd Neutron_Reflect/

/pscratch/sd/q/qubri000/Neutron_Reflect


In [ ]:
pip install --upgrade pip

In [ ]:
pip install ray==2.9.3

In [ ]:
pip install -U tensorboardx

In [ ]:
pip install optuna

In [ ]:
pip install fsspec

In [ ]:
pip install pyarrow

In [ ]:
# pip install tensorboard==2.12.0

In [ ]:
# pip install -U protobuf==3.19.6

In [ ]:
pip install gdown

In [ ]:
pip install seaborn

In [ ]:
pip install -U ipywidgets

In [ ]:
pip install plotly

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import numpy as np
import data_preparation as dpre
import io
import os
# import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
%matplotlib inline
import seaborn as sea

from ray import air, tune
from ray.air import session
from ray.tune.schedulers import ASHAScheduler
from ray.air.checkpoint import Checkpoint


In [4]:
#Loading in NR and SLD curves


curves_nr = np.load('../NR-SLD_CurvesPoly11.npy')
curves_SLD = np.load('../SLD_CurvesPoly11.npy')
curves_nr.shape, curves_SLD.shape


((50000, 2, 308), (50000, 2, 900))

In [ ]:
#plot NR curves

for x in range(curves_nr.shape[0]):
  plt.plot(curves_nr[x][0],curves_nr[x][1])
plt.xscale('log')
plt.yscale('log')

In [5]:
#normalize NR curves

curves_nr = np.log10(curves_nr)

x_points = []
y_points = []
for curve in curves_nr:
    x_points.append(curve[0])
    y_points.append(curve[1])
min_valX = float('inf')
min_valY = float('inf')
max_valX = -float('inf')
max_valY = -float('inf')
for i in range(len(y_points)):
    min_valX = min(min(x_points[i]), min_valX)
    min_valY = min(min(y_points[i]), min_valY)
    max_valX = max(max(x_points[i]), max_valX)
    max_valY = max(max(y_points[i]), max_valY)
for i in range(len(y_points)):
    for j in range(len(y_points[0])):
        x_points[i][j] -= min_valX
        y_points[i][j] -= min_valY
        x_points[i][j] /= (max_valX - min_valX)
        y_points[i][j] /= (max_valY - min_valY)
curves_nr2 = []
for i in range(len(y_points)):
    curves_nr2.append([x_points[i],y_points[i]])
curves_nr2 = np.stack(curves_nr2)
curves_nr2.shape
# for x in range(curves_nr.shape[0]):
#   plt.plot(curves_nr[x][0],curves_nr[x][1])

(50000, 2, 308)

In [ ]:
#plot SLD curves

for x in range(curves_nr.shape[0]):
  plt.plot(curves_SLD[x][0],curves_SLD[x][1])
# plt.xscale('log')
# plt.yscale('log')
print(curves_SLD[0])

In [6]:
#normalize SLD curves
x_points = []
y_points = []
for curve in curves_SLD:
    x_points.append(curve[0])
    y_points.append(curve[1])
min_valX = float('inf')
min_valY = float('inf')
max_valX = -float('inf')
max_valY = -float('inf')
for i in range(len(y_points)):
    min_valX = min(min(x_points[i]), min_valX)
    min_valY = min(min(y_points[i]), min_valY)
    max_valX = max(max(x_points[i]), max_valX)
    max_valY = max(max(y_points[i]), max_valY)
for i in range(len(y_points)):
    for j in range(len(y_points[0])):
        x_points[i][j] -= min_valX
        y_points[i][j] -= min_valY
        x_points[i][j] /= (max_valX - min_valX)
        y_points[i][j] /= (max_valY - min_valY)
curves_SLD2 = []
for i in range(5000):
    curves_SLD2.append([x_points[i],y_points[i]])
curves_SLD2 = np.stack(curves_SLD2)
# for x in range(curves_nr.shape[0]):
#   plt.plot(curves_SLD[x][0],curves_SLD[x][1])
# plt.xscale('log')
# plt.yscale('log')
print(curves_SLD[0])

[[0.         0.0007949  0.0015898  ... 0.71302488 0.71381978 0.71461468]
 [0.36503044 0.36503044 0.36503044 ... 0.08686069 0.08665173 0.08647615]]


In [7]:
#Make CNN class

class CNN(nn.Module):
    def __init__(self, num_layers):
        super(CNN, self).__init__()
        self.layers = nn.ModuleList()
        addition = 255/num_layers
        curr = 1
        for hdim in range(num_layers-1):
            self.layers.append(nn.Conv1d(int(curr+0.5), int(curr+addition+0.5),51,padding=25))
            self.layers.append(nn.BatchNorm1d(int(curr+addition+0.5)))
            self.layers.append(nn.ReLU(True))
            curr += addition
        self.layers.append(nn.Conv1d(int(curr+0.5), 256, 51, padding=25))
        self.linear1 = nn.Linear(256*308,2*308)
    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        x = x.reshape(x.shape[0], -1)
        x = self.linear1(x)
        x = x.reshape(-1, 2, 308)
        return torch.sigmoid(x)

In [8]:
### Define the loss function
loss_fn = torch.nn.MSELoss()
#learning rate
lr= 0.00001

### Set the random seed for reproducible results
torch.manual_seed(0)

#make CNN
Model = CNN(num_layers=5)
params_to_optimize = Model.parameters()

optimizer = torch.optim.Adam(params_to_optimize, lr=lr, weight_decay=1e-5)
# Check if the GPU is available
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(f'Selected device: {device}')


Selected device: cuda


In [9]:
Model.to(device)

CNN(
  (layers): ModuleList(
    (0): Conv1d(1, 52, kernel_size=(51,), stride=(1,), padding=(25,))
    (1): BatchNorm1d(52, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv1d(52, 103, kernel_size=(51,), stride=(1,), padding=(25,))
    (4): BatchNorm1d(103, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): Conv1d(103, 154, kernel_size=(51,), stride=(1,), padding=(25,))
    (7): BatchNorm1d(154, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU(inplace=True)
    (9): Conv1d(154, 205, kernel_size=(51,), stride=(1,), padding=(25,))
    (10): BatchNorm1d(205, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): ReLU(inplace=True)
    (12): Conv1d(205, 256, kernel_size=(51,), stride=(1,), padding=(25,))
  )
  (linear1): Linear(in_features=78848, out_features=616, bias=True)
)

In [10]:
## Training function
def fit(Model, dataloader, loss_fn, optim):
    device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
    Model.train().to(device)
    train_loss = []
    for data,label in dataloader:
        img = data
        img = img.to(device)
        label = label.to(device)
        decoded_img = Model(img)
        # print(decoded_img.shape)
        # decoded_img = decoded_img.reshape(-1,2, 308)
        loss = loss_fn(decoded_img, label)
        optim.zero_grad()
        loss.backward()
        optim.step()
        train_loss.append(loss.cpu().detach().numpy())
        # print(label, "hello")
    return np.mean(train_loss)

In [11]:
### Valid function
def val(Model, dataloader, loss_fn):
    device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
    Model.eval().to(device)
    with torch.no_grad(): 
        list_decoded_img = []
        list_img = []
        for  data, label in dataloader:
            img = data
            # img = img.view(img.size(0), -1).to(device) 
            # img = img[:,np.newaxis,:].to(device) 
            img = img.to(device) 
            label = label.to(device)
            decoded_img = Model(img)
            # decoded_img = decoded_img.reshape(-1,2, 308)
            list_decoded_img.append(decoded_img.cpu())
            list_img.append(label.cpu())
        list_decoded_img = torch.cat(list_decoded_img)
        list_img = torch.cat(list_img)
        # print(list_img, list_decoded_img, "vaLLLLL FUNCTION")
#         for i in range(len(list_img)):
            
        val_loss = loss_fn(list_decoded_img, list_img)
    return val_loss.data

In [12]:
#data split function

def load_data(batch_size, size_split):
    R = curves_nr[:5000,1]
    R_m = R[:,np.newaxis,:]
    # Q = curves_SLD[:,1]
    # Q_m = Q[:, np.newaxis,:]
    xtrain, ytrain, xval, yval, xtest, ytest = \
    dpre.split_input_arrays(R_m,curves_SLD2, size_split=size_split)
    #Prepare data files, continuation
    train_dataset, valid_dataset, test_dataset, train_loader, valid_loader, test_loader = \
    dpre.get_dataloaders_fromsplitarrays(xtrain,ytrain,xval,yval,xtest,ytest,batch_size=batch_size)

    return train_loader, valid_loader, test_loader

In [13]:
train_loader, valid_loader, test_loader = load_data(32, 0.9)


xtrain.shape, ytrain.shape, xval.shape, yval.shape, xtest.shape, ytest.shape
(4500, 1, 308) (4500, 2, 900) (250, 1, 308) (250, 2, 900) (250, 1, 308) (250, 2, 900)


In [14]:
def train_cifar(config):
    net = CNN(num_layers=config["num_layers"])

    device = "cpu"
    if torch.cuda.is_available():
        device = "cuda:0"
        if torch.cuda.device_count() > 1:
            net = nn.DataParallel(net)
    net.to(device)

    criterion = nn.MSELoss()
    optimizer = optim.Adam(net.parameters(), lr=config["lr"], weight_decay=config["weight_decay"])
    # To restore a checkpoint, use `session.get_checkpoint()`.
    loaded_checkpoint = session.get_checkpoint()
    if loaded_checkpoint:
        with loaded_checkpoint.as_directory() as loaded_checkpoint_dir:
           model_state, optimizer_state = torch.load(os.path.join(loaded_checkpoint_dir, "checkpoint.pt"))
        net.load_state_dict(model_state)
        optimizer.load_state_dict(optimizer_state)

    for epoch in range(10):  # loop over the dataset multiple times
        running_loss = fit(net,  train_loader, criterion, optimizer)
        # Validation loss
        val_loss = val(net, valid_loader, criterion)

        # Here we save a checkpoint. It is automatically registered with
        # Ray Tune and can be accessed through `session.get_checkpoint()`
        # API in future iterations.
        os.makedirs("my_model", exist_ok=True)
        torch.save(
            (net.state_dict(), optimizer.state_dict()), "my_model/checkpoint.pt")
        checkpoint = Checkpoint.from_directory("my_model")
        session.report({"loss": (val_loss.item()), "training loss": running_loss}, checkpoint=checkpoint)
    print("Finished Training")

In [15]:
def test_best_model(best_result):
    best_trained_model = CNN(num_layers=best_result.config["num_layers"])
    device = "cuda:0" if torch.cuda.is_available() else "cpu"
    best_trained_model.to(device)

    checkpoint_path = os.path.join(best_result.checkpoint.to_directory(), "checkpoint.pt")
    
    model_state, optimizer_state = torch.load(checkpoint_path)
    best_trained_model.load_state_dict(model_state)

    testLoss = val(best_trained_model, test_loader, loss_fn)


    print("Best trial test set accuracy: {}".format(testLoss.item()))

In [ ]:
def main(num_samples=10, max_num_epochs=10, gpus_per_trial=1):
    config = {
    "lr": tune.loguniform(1e-5, 1e-3), "weight_decay": tune.loguniform(1e-5, 1e-3), "num_layers":tune.choice([14, 15,16,17,18])
    }
    scheduler = ASHAScheduler(
        max_t=max_num_epochs,
        grace_period=1,
        reduction_factor=2)
    
    tuner = tune.Tuner(
        tune.with_resources(
            tune.with_parameters(train_cifar),
            resources={"cpu": 1, "gpu": gpus_per_trial}
        ),
        tune_config=tune.TuneConfig(
            metric="loss",
            mode="min",
            scheduler=scheduler,
            num_samples=num_samples,
        ),
        param_space=config,
        run_config=air.RunConfig(local_dir="$PSCRATCH", name="test_experiment")
    )
    results = tuner.fit()
    best_result = results.get_best_result()

    print("Best trial config: {}".format(best_result.config))
    print("Best trial final validation loss: {}".format(
        best_result.metrics["loss"]))
    print("Best trial final training loss: {}".format(
        best_result.metrics["training loss"]))
    test_best_model(best_result)

main(num_samples=5, max_num_epochs=50)

/global/homes/q/qubri000/.local/perlmutter/pytorch2.0.1/lib/python3.9/site-packages/ray/air/config.py:803: UserWarning: Setting a `RunConfig.local_dir` is deprecated and will be removed in the future. If you are not using remote storage,set the `RunConfig.storage_path` instead. Otherwise, set the`RAY_AIR_LOCAL_CACHE_DIR` environment variable to control the local cache location.
  warnings.warn(
2024-03-12 18:09:31,965	INFO worker.py:1621 -- Started a local Ray instance.
